Free Translate (translate.google.com)

In [ ]:
import yaml
import json
from pathlib import Path
from googletrans import Translator

In [ ]:
# Load configuration
with open("../config.yaml") as f:
    config = yaml.safe_load(f)

# Set up project paths
project_root = Path.cwd().parent
PARALLEL_DATA_DIR = project_root / config["PARALLEL_DATA_DIR"]

# Progress bar format
PROGRESS_BAR_FORMAT = "{desc:<25}{percentage:3.0f}%|{bar:20}{r_bar}"


In [ ]:
for lang_code, lang_config in config["LANGUAGES"].items():
    parallel_sents_file = PARALLEL_DATA_DIR / f"{lang_code}-en_data.jsonl"

    with open(parallel_sents_file, "r", encoding="utf-8") as file:
        sentences = [json.loads(line) for line in file]

    if not sentences:
        continue

    

    # Extract translations
    translated_sentences = [
        res["response"]["body"]["choices"][0]["message"]["content"]
        for res in batch_responses[lang_code]
    ]

    assert len(translated_sentences) == len(lang_sents), (
        f"Mismatch: {len(translated_sentences)} translations vs {len(lang_sents)} source sentences"
    )

    # Write parallel data
    with open(parallel_sents_file, "w", encoding="utf-8") as outfile:
        for target, source in zip(lang_sents, translated_sentences):
            outfile.write(
                json.dumps(
                    {
                        "target_text": target["text"],
                        "target_lang": lang_code,
                        "source_text": source,
                        "source_lang": "en",
                        "doc_id": target["doc_id"],
                        "sent_id": target["sent_id"],
                    },
                    ensure_ascii=False,
                )
                + "\n"
            )

    print(
        f"Created parallel data for {lang_code}: {len(translated_sentences)} sentence pairs"
    )
